<a href="https://colab.research.google.com/github/catPotat/rn-hg-gpt2-playground/blob/main/VN_corpus_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏁 Initialize

In [ ]:
!pip install transformers==4.4.2
!pip install wikiextractor
!pip install python-dotenv
try:
    from google.colab import output
    output.clear()
    from google.colab import drive
    drive.mount('/content/drive')
except: pass

Mounted at /content/drive


In [ ]:
%load_ext dotenv
%dotenv drive/MyDrive/.env.gdrive
import os
from tqdm.notebook import tqdm
import torch
import random

# Common

## Paths

In [ ]:
MODEL_NAME = "gpt2"
MODEL_SAVE_PATH = '/content/drive/MyDrive/models/VinaPT-0421/vinaPT-attempt-0405/'
TOK_PATH = MODEL_SAVE_PATH+'tokenizer-wordseg'
MODEL_LENGTH=1024

## Segmenter

In [ ]:
!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install vncorenlp
from vncorenlp import VnCoreNLP
# annotator = VnCoreNLP('/content/VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx500m')

def word_segment(text):
    word_segmented_text = annotator.tokenize(text) 
    sentences = []
    for sentence in word_segmented_text:
        sentences.append(' '.join(sentence))
    return ' '.join(sentences)

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 212, done.
remote: Total 212 (delta 0), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (212/212), 214.21 MiB | 41.96 MiB/s, done.
Resolving deltas: 100% (76/76), done.
     |████████████████████████████████| 2.7MB 15.9MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=710df8705065f307d4f1f16c19a8100fb1e3bb133f6dd96c969291c67e71b887
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


## 🪓 Tokenizer

### Train

In [ ]:
root, dirs, files = next(os.walk('/content/drive/MyDrive/datasets/vietnamese-corpus/vnNewsCorpus-vnCoreNlp-treated', topdown=False))
files = list( map(lambda x: os.path.join(root, x), files) )
files.append('/content/drive/MyDrive/datasets/vietnamese-corpus/viWikitext-VnCoreNlp-treated/9110120.txt')
files

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tok_trainer = ByteLevelBPETokenizer()
tok_trainer.train(
    files=files,
    vocab_size=50257,
    min_frequency=10,
    special_tokens=[
        '<|endoftext|>',
    ]
)
!mkdir -p 'tok/'
tok_trainer.save_model('tok/', "igptkinda")
!mkdir -p $TOK_PATH
!cp 'tok/igptkinda-vocab.json' $TOK_PATH/vocab.json
!cp 'tok/igptkinda-merges.txt' $TOK_PATH/merges.txt

### Pre trained

In [ ]:
from transformers import GPT2Tokenizer, GPT2TokenizerFast, AutoTokenizer
tokenizer = GPT2TokenizerFast.from_pretrained(TOK_PATH, model_max_length=MODEL_LENGTH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length

1024

# 📰 vn news corpus

https://github.com/binhvq/news-corpus

https://github.com/vncorenlp/VnCoreNLP

### Overview

In [ ]:
i = 0
with open('/content/drive/MyDrive/datasets/vietnamese-corpus/corpus-full.txt', 'r', encoding="utf8") as f:
    for line in f:
        i+=1
i

111274300

## Segment

In [ ]:
annotator = VnCoreNLP('/content/VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx500m')

dump = ""
i = 0
with open('/content/drive/MyDrive/datasets/vietnamese-corpus/corpus-full.txt', 'r', encoding="utf8") as f:
    # for line in tqdm(f, total=111274300):
    for line in f:
        i += 1
        if i <= 24_000_000:
            continue
        dump += word_segment(line) + '\n'
        if i % 1_000_000 == 0:
            with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/vnNewsCorpus-vnCoreNlp-treated/{i}.txt', 'w+', encoding="utf8") as f:
                f.write(dump)
            dump = ""
            print(i)
with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/vnNewsCorpus-vnCoreNlp-treated/{i}.txt', 'w+', encoding="utf8") as f:
    f.write(dump)
del dump
print(i)

### Check for completion

In [ ]:
total = [ (x+1)*811003 for x in range(200) ]
total

In [ ]:
from tqdm.notebook import tqdm
ls = []
i = 0
corpus_path = '/content/drive/MyDrive/datasets/vietnamese-corpus/'
for root, dirs, files in os.walk(corpus_path, topdown=False):
    for j, name in enumerate(files):
        cpoint = name.split('.')[0]
        if not cpoint.isnumeric():
            continue
        ls.append( cpoint )
        i += 1

In [ ]:
unfinished = []
for item in total:
    if str(item) not in ls:
        unfinished.append(item)
unfinished

[63258234]

In [ ]:
ls

## Trimmer

In [ ]:
import psutil
import gc
RAM = psutil.virtual_memory().total
dump = ""
i = 0
with open(os.path.join(CORPUS_PATH, 'corpus-full-0.2.txt'), 'r', encoding="utf8") as f:
    # for line in tqdm(f, total=162200680):
    for line in f:
        l = len( line.split() )
        if l == 7:
            # continue
            print(line)
        # dump += line
        # i += 1
        # if i%5_000_000==0 and psutil.virtual_memory().available / RAM > 0.8:
        #     print(i)
        #     with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/raw-trimmed/line{i}.txt', 'w+', encoding="utf8") as f:
        #         f.write(dump)
        #     dump = ""
        #     gc.collect()

## Pre-tokenize

In [ ]:
import random
root, dirs, files = next(os.walk('/content/drive/MyDrive/datasets/vietnamese-corpus/vnNewsCorpus-vnCoreNlp-treated', topdown=False))
files = list( map(lambda x: os.path.join(root, x), files) )
ip_ids_line = ""
for i, path in tqdm( enumerate(files, start=1), total = len(files) ):
    if i < 0: continue
    to_tokenize = ""
    with open(path, 'r', encoding="utf8") as f:
        for line in f:
            l = len( line.split() )
            if l <= 7:
                continue
                print(line)
            # continue
            if l > 300:
                ip_ids = tokenizer(line, truncation=True, padding=False)['input_ids']
                ip_ids_line += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
                # if random.random() < 0.001:
                #     print( f"1line{len(ip_ids)}" )
                continue
            if len( to_tokenize.split() ) < 300:
                to_tokenize += line
                continue
            to_tokenize += line
            ip_ids = tokenizer(to_tokenize, truncation=True, padding=False)['input_ids']
            # if len(ip_ids) == 1024:
            #     print( "a" )
            # if random.random() < 0.01:
            #     print( f"manyline{len(ip_ids)}" )
                # print(to_tokenize)
            to_tokenize = ""
            ip_ids_line += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
        if i % 10 == 0:
            if to_tokenize:
                ip_ids = tokenizer(to_tokenize, truncation=True, padding=False)['input_ids']
                ip_ids_line += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
            with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/pretokenized-0405/{i}.txt', 'w+') as out:
                out.write(ip_ids_line)
            ip_ids_line = ""
with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/pretokenized-0405/{i}.txt', 'w+') as out:
    out.write(ip_ids_line)

KeyboardInterrupt: ignored

# 🌐 viWikitext

In [ ]:
!python -m wikiextractor.WikiExtractor /content/drive/MyDrive/datasets/vietnamese-corpus/viwiki-20210401-pages-meta-current.xml

### Segment

In [ ]:
import html

annotator = VnCoreNLP('/content/VnCoreNLP/VnCoreNLP-1.1.1.jar', annotators='wseg', max_heap_size='-Xmx500m')

dump = ""
i = 0
for root, dirs, files in os.walk('/content/text', topdown=False):
    print(root)
    for name in tqdm( files ):
        with open(os.path.join(root, name), 'r', encoding="utf8") as f:
            
            to_tokenize = ""
            for line in f:
                if len( line.split() ) <= 7:
                    continue
                    print(line)
                # continue
                if len( line.split() ) > 300:
                    ip_ids = tokenizer(line, truncation=True, padding=False)['input_ids']
                    ip_ids_line += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
                    # print( len(ip_ids) )
                    # if len(ip_ids) == 1024:
                    #     print( "a" )
                    continue
                if len( to_tokenize.split() ) < 300:
                    to_tokenize += line
                    continue
                to_tokenize += line
                ip_ids = tokenizer(to_tokenize, truncation=True, padding=False)['input_ids']
                # print( len(ip_ids) )
                # if len(ip_ids) == 1024:
                #     print( "a" )
                # print( to_tokenize )
                to_tokenize = ""
                ip_ids_line += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
            if i % 20 == 0:
                with open(os.path.join(CORPUS_PATH, 'pre-tokenized', f'{i}.txt'), 'w+') as out:
                    out.write(ip_ids_line)
                ip_ids_line = ""


            for line in f:
                i += 1
                line = html.unescape(line)
                line = line.strip()
                if not line:
                    continue
                if line.startswith('<'):
                    dump += line + '\n'
                else:
                    dump += word_segment(line) + '\n'
with open(f'/content/drive/MyDrive/datasets/vietnamese-corpus/viWikitext-VnCoreNlp-treated/{i}.txt', 'w+', encoding="utf8") as f:
    f.write(dump)
del dump
print(i)

### Pre-tokenize

In [ ]:
import time, random

ip_ids_line1 = ""
ip_ids_line2 = ""
ip_ids_line3 = ""

skip_title = False
longest_len = 0
SLIDE_SIZE = 500
SLIDE_STRIDE = 400
article = "0 "
to_tokenize = ""
to_tokenize2 = ""
with open("/content/drive/MyDrive/datasets/vietnamese-corpus/viWikitext-VnCoreNlp-treated/9110120.txt", 'r', encoding="utf8") as f:
    for i, line in enumerate(f):
        if skip_title:
            skip_title = False
            continue
        if line.startswith("<doc"):
            skip_title = True
            continue
        elif line.startswith("</doc"):
            splitted = article.split()
            l = len(splitted)
            '''
            if l > 10 and l<= 45:
                # if random.random() < 0.001:
                #     print(article)
                to_tokenize2 += article.strip()+'\n'
                l2 = len(to_tokenize2.split())
                if l2 > 500:
                    ip_ids = tokenizer(to_tokenize2, truncation=True, padding=False)['input_ids']
                    ip_ids_line1 += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
                    to_tokenize2 = ""
                    if random.random() < 0.0001:
                        print(len(ip_ids))
            
            if l > 45 and l-1 <= SLIDE_SIZE:
                to_tokenize += article.strip()+'\n'
                l2 = len(to_tokenize.split())
                if l2 > 300:
                    ip_ids = tokenizer(to_tokenize, truncation=True, padding=False)['input_ids']
                    ip_ids_line2 += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
                    to_tokenize = ""
                    if random.random() < 0.0001:
                        print(len(ip_ids))
            '''
            if l-1 > SLIDE_SIZE:
                pos = 0
                sliding = True
                while True:
                    window = splitted[pos:pos+SLIDE_SIZE]
                    ip_ids = tokenizer(' '.join(window), truncation=True, padding=False)['input_ids']
                    # print(' '.join(window))
                    # time.sleep(.1)
                    ip_ids_line3 += ' '.join( map(lambda x: str(x), ip_ids) ) + '\n'
                    if not sliding: break
                    pos += SLIDE_STRIDE
                    if pos+SLIDE_SIZE > l-1:
                        pos = l-1-SLIDE_SIZE
                        sliding = False
                    if random.random() < 0.001:
                        print(len(ip_ids))
            
            article = ""
        elif line.startswith('<'):
            continue
        else:
            if len(line.split()) < 7:
                continue
            article += line

# with open('/content/drive/MyDrive/datasets/vietnamese-corpus/pretokenized-0405/wikitext10-45.txt', 'w+') as out:
#     out.write(ip_ids_line1)
# with open('/content/drive/MyDrive/datasets/vietnamese-corpus/pretokenized-0405/wikitext46-300.txt', 'w+') as out:
#     out.write(ip_ids_line2)
with open('/content/drive/MyDrive/datasets/vietnamese-corpus/pretokenized-0405/wikitext301v2.txt', 'w+') as out:
    out.write(ip_ids_line3)

858
778
922
762
799
771
920
798
709
814
749
814
822
913
902
726
730
760
838
780
872
825
846
736
831
951
858
804
898
730
826
760
776
811
783
800
789
790
802
725
786
755
789
824
733
755
693
792
743
753
819
881
745
703
707
777
759
686
807
797
830
894
817
918
873
776
780
839
894
805
813
795
774
834
733
862
800
695
785
750
797
813
754
702
833
746
693
914
1024
764
684
828
759
865
875
770
752
1003
776
868
744
870
785
850
840
900
759
841
875
806
734
834
770
892
822
840
740
763
864
796
728
923
899
763
832


In [ ]:
with open(os.path.join('/content/drive/MyDrive/datasets/vietnamese-corpus/pre-tokenized', 'wikitext301.txt'), 'r') as out:
    for line in out:
        len_ = len(line.strip().split())
        if random.random() < 0.01:
            print(len_)
        # if len_ >= 1024:
        #     print(len_)

## Visualize

In [ ]:
RANGE = 200
entry_len = {x: 0 for x in range(RANGE+1)}

skip_title = False
longest_len = 0
with open("/content/drive/MyDrive/datasets/vietnamese-corpus/viWikitext-VnCoreNlp-treated/9110120.txt", 'r', encoding="utf8") as f:
    article = ""
    for i, line in enumerate(f):
        if skip_title:
            skip_title = False
            continue
        if line.startswith("<doc"):
            skip_title = True
            continue
        elif line.startswith("</doc"):
            l = len(article.split())
            entry_len[l if l<=RANGE else RANGE] += 1
            if l > 10 and l < 20 and l!=0 and article.endswith(':\n'):
                print(article)
                # break
            article = ""
        else:
            article += line

        # if l > longest_len:
        #     longest_len = l
        #     print(f'index: {i}')
        #     print(f'len: {l}')
        #     print(line)


import matplotlib.pyplot as plt
plt.bar(range(len(entry_len)), list(entry_len.values()))
plt.show()

# test

In [ ]:
from transformers import pipeline, set_seed

generator = pipeline(
    'text-generation',
    model=\
#         os.path.join(CHECKPOINT_PATH, 'checkpoint-80000'),
        # model,
        'gpt2-large',
    tokenizer=\
#         tokenizer
        'gpt2-large'
)
# set_seed(6110)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias', 'h.24.attn.masked_bias', 'h.25.attn.masked_bias', 'h.26.attn.masked_bias', 'h.27.attn.masked_bias', 'h.28.attn.masked_bias', 'h.29.attn.masked_bias', 'h.30.attn.masked_bias', 'h.31.attn.masked_bias', 'h.32.attn.masked_bias', 'h.33.attn.masked_bias', 'h.34.attn.masked_bias', 'h.35.attn.masked_bi

In [ ]:
result = generator("", max_length=1028, num_return_sequences=3)
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: ignored

In [ ]:
result = generator("", max_length=512, num_return_sequences=3)
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The Federal Open Market Committee announced its decision to keep its existing program of bond buying active as it considers the composition of its asset purchase portfolio.\n\nThe decision to maintain a balance sheet purchases appears well-timed given market conditions, with higher short and long term yields than recent levels, the announcement noted at its policy meeting held Tuesday evening in Washington, D.C.\n\n"We look forward to continuing to build on today\'s progress and continuing our effort to advance our goal of maximum employment and underemployment," said Minneapolis Fed President Narayana Kocherlakota.\n\nA two-year portfolio of $75-billion of Treasury securities with a goal of achieving a multiyear low fed funds rate below the 2 percent target will remain under review, the bank\'s statement noted. The Bank of Canada and the European Central Bank have announced similar asset purchases.\n\n"Today\'s announcement does not mean they will be done," Kocher

In [ ]:
print( result[1]['generated_text'] )

In the past couple days, the news about Donald Trump has come pouring in. It has been quite a shocker, but one that may have some major ramifications in how the people view this country. After all, who else can pull that sort of stunts as shamelessly as Trump?

Trump's first 100 days as the 45th president began Sunday, when he issued his first executive order and signed 12 executive orders in three days, making good on his promise to build a wall on the southern border, deport all 11 million "criminal aliens" and ban all Muslims from entering the U.S. He began by calling out Washington D.C., and his first act in office was to order to begin the implementation of the first version of his plan to build his wall.

But Trump's antics don't end there. He has already taken the first steps toward the second revision of his Muslim ban. President Trump directed the Department of Homeland Security and the Department of Defense to start the construction of the most comprehensive Muslim travel ban